In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
data_type_list = ['source_list', 'cam_param', 
                  'world_3d', 'cam_3d', 'img_2d', 
                  #'img_3d', 'scale_factor', 'img_25d', 
                  #'cam_3d_canonical_same_z', 'img_2d_canonical_same_z', 
                  #'cam_3d_canonical_same_dist', 'img_2d_canonical_same_dist',
                  'cam_3d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5']
overwrite_list = []
h36m = load_data_dict('h36m', data_type_list, overwrite_list, verbose=False)
fit3d = load_data_dict('fit3d', data_type_list, overwrite_list, verbose=False)
_3dhp = load_data_dict('3dhp', data_type_list, overwrite_list, verbose=False)

In [3]:
h36m.keys()

dict_keys(['source_list', 'cam_param', 'world_3d', 'cam_3d', 'img_2d', 'cam_3d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5'])

In [40]:
for dataset_name in ['h36m', 'fit3d', '3dhp']:
    if dataset_name == 'h36m': data = h36m.copy()
    elif dataset_name == 'fit3d': data = fit3d.copy()
    elif dataset_name == '3dhp': data = _3dhp.copy()
    img_2d_canonicals_fixed_dist_5 = []
    cam_params = []
    for source in data['source_list']:
        if dataset_name == '3dhp':
            if 'TS' not in source: continue
        subject, cam_id, action = split_source_name(source, dataset_name)
        img_2d_canonicals_fixed_dist_5 += list(data[f'img_2d_canonical_fixed_dist_5'][subject][action][cam_id]) 
        cam_param = data['cam_param'][subject][action][cam_id]
        num_frames = cam_param['num_frames']
        cam_params += [cam_param] * num_frames
    img_2d_canonicals_fixed_dist_5 = np.array(img_2d_canonicals_fixed_dist_5)
    cam_params = np.array(cam_params)

    W_array = np.array([cam_param['W'] for cam_param in cam_params])
    H_array = np.array([cam_param['H'] for cam_param in cam_params])
    fx_array = np.array([cam_param['intrinsic'][0][0] for cam_param in cam_params])
    fy_array = np.array([cam_param['intrinsic'][1][1] for cam_param in cam_params])
    cx_array = np.array([cam_param['intrinsic'][0][2] for cam_param in cam_params])
    cy_array = np.array([cam_param['intrinsic'][1][2] for cam_param in cam_params])
    img_2d_canonicals_fixed_dist_5_norm = img_2d_canonicals_fixed_dist_5.copy()
    img_2d_canonicals_fixed_dist_5_norm *= 2
    img_2d_canonicals_fixed_dist_5_norm -= np.concatenate([W_array[:, None, None], H_array[:, None, None]], axis=2)
    img_2d_canonicals_fixed_dist_5_norm /= W_array[:, None, None]

    x_max = np.max(img_2d_canonicals_fixed_dist_5_norm[:, :, 0], axis=1)
    x_min = np.min(img_2d_canonicals_fixed_dist_5_norm[:, :, 0], axis=1)
    y_max = np.max(img_2d_canonicals_fixed_dist_5_norm[:, :, 1], axis=1)
    y_min = np.min(img_2d_canonicals_fixed_dist_5_norm[:, :, 1], axis=1)

    print((x_max - x_min).mean(), (y_max - y_min).mean(), fx_array.mean(), fy_array.mean(), cx_array.mean(), cy_array.mean())


0.26305292292163057 0.6186447454669062 1147.344289910138 1146.2364022588524 514.043540276127 506.70027037310103
0.29063875117540067 0.6513030411792289 1062.1287424441518 1056.9917459263422 465.8927253875467 453.59153766308646
0.21050821172529957 0.40780679681098625 1506.3911601366224 1506.141669979055 1014.764274844631 1026.6909053252234
